In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('always')
from sklearn.model_selection import train_test_split
import shapely.speedups
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
import re
from itertools import combinations
import random
import joblib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
#
np.random.seed(1)
tf.random.set_seed(1)

c:\Users\iankl\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
c:\Users\iankl\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
c:\Users\iankl\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:43: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.F

In [3]:
def plot_regression(modelo,x,y,title="",xlabel="x",ylabel="y"):
    plt.figure()
    
    plt.plot(x,y,"o",label="Valores verdaderos")
    plt.plot(x,modelo.predict(x),"x",label="Valores estimados")
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [4]:
viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_test.csv').copy()
viviendas = pd.concat([viviendas_train, viviendas_test])

In [5]:
viviendas = pd.get_dummies(viviendas, columns = ["property_type"])
viviendas = pd.get_dummies(viviendas, columns = ["barrio"])

In [6]:
viviendas_train, viviendas_test = train_test_split(viviendas, test_size=0.2, random_state=2)

definimos las columnas que utilizaremos para entrenar las redes

In [7]:
columnas_predictoras=viviendas_train.columns.to_list()
for variable in ['property_price', 'start_date', 'end_date', 'property_title', 'geometry']:
    columnas_predictoras.remove(variable)
columnas_predictoras

['latitud',
 'longitud',
 'property_rooms',
 'property_bedrooms',
 'property_surface_total',
 'property_surface_covered',
 'property_type_Casa',
 'property_type_Departamento',
 'property_type_PH',
 'barrio_Abasto',
 'barrio_Agronomía',
 'barrio_Almagro',
 'barrio_Balvanera',
 'barrio_Barracas',
 'barrio_Barrio Norte',
 'barrio_Belgrano',
 'barrio_Boca',
 'barrio_Boedo',
 'barrio_Caballito',
 'barrio_Catalinas',
 'barrio_Centro / Microcentro',
 'barrio_Chacarita',
 'barrio_Coghlan',
 'barrio_Colegiales',
 'barrio_Congreso',
 'barrio_Constitución',
 'barrio_Flores',
 'barrio_Floresta',
 'barrio_Las Cañitas',
 'barrio_Liniers',
 'barrio_Mataderos',
 'barrio_Monserrat',
 'barrio_Monte Castro',
 'barrio_Nuñez',
 'barrio_Once',
 'barrio_Palermo',
 'barrio_Parque Avellaneda',
 'barrio_Parque Centenario',
 'barrio_Parque Chacabuco',
 'barrio_Parque Chas',
 'barrio_Parque Patricios',
 'barrio_Paternal',
 'barrio_Pompeya',
 'barrio_Puerto Madero',
 'barrio_Recoleta',
 'barrio_Retiro',
 'barrio_S

## RANDOMIZADOR


Para ontrenar las redes neuronales utilizaresmos un random_search para optimizar los hiperparametros

In [8]:
a = ['property_price', 'start_date', 'end_date', 'property_title', 'geometry']
b = viviendas_train.loc[:,a]
b

,property_price,start_date,end_date,property_title,geometry
9922,149000.0,2021-02-17,2021-05-18,2 ambientes en extraordinarias condiciones.OPO...,POINT (-58.4154363 -34.5968093)
14985,112000.0,2021-05-18,2021-06-05,Departamento 2 amb 45m2 en Venta - Edificio Be...,POINT (-58.4711685 -34.5486901)
47162,157000.0,2021-02-05,2021-04-29,Departamento - Recoleta,POINT (-58.4074484 -34.5888169)
15807,85000.0,2021-04-30,2021-06-05,Departamento - Barrio Norte,POINT (-58.3943446 -34.598379)
19883,125000.0,2021-05-29,2021-06-19,Wel. Venta.Departamento.2 ambientes.Colegiales,POINT (-58.4487278 -34.5743883)
...,...,...,...,...,...
10131,598000.0,2021-08-19,2021-11-04,Piso de categoria de 4 Ambientes mas escritori...,POINT (-58.4320439 -34.5672732)
31019,110000.0,2021-12-13,2022-01-27,Departamento - Villa del Parque,POINT (-58.4977 -34.6110437)
44566,120000.0,2021-09-06,2021-09-15,HOY SIN EXPENSAS EN MARMOL Y BELGRANO - T/A NU...,POINT (-58.4259572 -34.6168581)
72173,108900.0,2021-03-31,2021-04-25,DIVINO UNICO Monoamb Apaisado VILLA URQUIZA,POINT (-58.4891098738 -34.5711417284)


In [9]:
#al instanciar un ModeloRedes crea y entrena una red neuronal con los hiperparametros pasados
class ModeloRedes:
    def __init__(self, dataset_train, dataset_test, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size):
        self.variables_predictoras = variables_predictoras
        self.cantidad_primera_capa = cantidad_de_primera_capa
        self.funcion_activacion = funcion_activacion
        self.optimizador = optimizador
        self.learning_rate = learning_rate
        self.resultado = None
        self.metrica = metrica
        self.epoch = epoch
        self.batch_size = batch_size
        self.modelo = None

        
        self.x_test = dataset_test.loc[:,variables_predictoras]
        self.y_test = dataset_test.loc[:,'property_price']
        self.x_train = dataset_train.loc[:,variables_predictoras]
        self.y_train = dataset_train.loc[:,'property_price']
        self.x_test_original = None
        self.x_train_original = None
        self.precio_maximo = max(max(dataset_test['property_price']), max(dataset_train['property_price']))
        self.escalar_datasets()
        self.entrenar_modelo()
    
    def escalar_datasets(self):
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(self.x_train))
        self.x_train_original = self.x_train
        self.x_test_original = self.x_test
        self.x_train=sscaler.transform(pd.DataFrame(self.x_train))
        self.x_test=sscaler.transform(pd.DataFrame(self.x_test))
            
        self.y_train = self.y_train / self.precio_maximo
        

    def entrenar_modelo(self):
    # Creo un modelo Red Neuronal
        d_in= len(self.variables_predictoras)
        d_out=1

        capas = [
            # input layer
            keras.layers.Dense(d_in,input_shape=(d_in,), activation= self.funcion_activacion)
        ]

        #inserta capas que van decreciendo en cantidad de neuronas
        for i in range(self.cantidad_primera_capa,2,-1):
            capas.append(keras.layers.Dense(i, activation= self.funcion_activacion ))

        capas.append(keras.layers.Dense(d_out, ))
        self.modelo = keras.Sequential(capas)

        self.modelo.compile(
        optimizer= self.optimizador(self.learning_rate), 
        loss=self.metrica, 
        metrics=[self.metrica], 
        )

        self.modelo.fit(self.x_train,self.y_train,epochs=self.epoch,batch_size=self.batch_size,verbose=False)
    
    def plotear(self):
        plot_regression(self.modelo,self.x_test,self.y_test,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        
    def testear_modelo(self):
        y_pred=self.modelo.predict(self.x_test)
        for i in y_pred:
            i[0] = i[0] * self.precio_maximo
        mse=mean_squared_error(self.y_test,y_pred)
        self.resultado = mse ** (1/2)
        return self.resultado
    
    def predecir(self, df):

        df_para_predecir =  df.loc[:,self.variables_predictoras]
        df_precios = df.loc[:,'property_price']
        
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(df_para_predecir))
        df_para_predecir=sscaler.transform(pd.DataFrame(df_para_predecir))

        
        y_pred=self.modelo.predict(df_para_predecir)
        for i in y_pred:
            i[0] = i[0] * self.precio_maximo

        mse=mean_squared_error(df_precios,y_pred)
        resultado = mse ** (1/2)
        #plot_regression(self.modelo,df_para_predecir,df_precios,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        return y_pred , resultado
    
    def info(self):
        print('\n','\n')
        print('\n',
        self.variables_predictoras , '\n',
        self.cantidad_primera_capa ,'\n',
        self.funcion_activacion ,'\n',
        self.optimizador ,'\n',
        self.learning_rate ,'\n',
        self.metrica ,'\n',
        self.epoch ,'\n',
        self.batch_size ,'\n')

definimos los conjuntos de hiperparametros que queremos que pruebe

In [10]:
columnas_importantes =  ['latitud', 'longitud', 'property_surface_total', 'property_surface_covered']
columnas_extras = []
for e in columnas_predictoras:
    if e not in columnas_importantes:
        columnas_extras.append(e)       

def agregar_columnas(posibles_variables_predictoras, columnas, cant_columnas):
            i = 0
            while (i < cant_columnas): 
                pos = random.randrange(i,len(columnas), 1)
                posibles_variables_predictoras.append(columnas[pos])
                columnas[i], columnas[pos] = columnas[pos], columnas[i]
                i += 1

posibles_cantidad_de_primera_capa = []
for i in range(1,20):
    posibles_cantidad_de_primera_capa.append(i)
                                        #vimos que los modelos buenos utilizaban "relu" asique dejamos de utilizar el resto dentro de random search
posibles_funcion_activacion = ["relu","tanh", "softmax", "sigmoid"]
posibles_metricas = ['mse', 'mae']

posibles_learning_rate = [0.1, 0.01]

In [11]:
def funcion_optimizadoraNADAM(learnigRate):
            return keras.optimizers.Nadam(learning_rate = learnigRate)

def funcion_optimizadoraRMSPROP(learnigRate):
            return keras.optimizers.RMSprop(learning_rate = learnigRate)

def funcion_optimizadoraADAMAX(learnigRate):
            return keras.optimizers.Adamax(learning_rate = learnigRate)

def funcion_optimizadoraADADELTA(learnigRate):
            return keras.optimizers.Adadelta(learning_rate = learnigRate)

posibles_optimizador = [funcion_optimizadoraNADAM, funcion_optimizadoraRMSPROP, funcion_optimizadoraADAMAX, funcion_optimizadoraADADELTA]

In [12]:
def obtener_posibles_variables_predictoras():
        posibles_variables_predictoras = []     
        cant_columnas_importantes = random.randrange(1,len(columnas_importantes)+1, 1)
        cant_columnas_extras = random.randrange(0,len(columnas_extras)+1, 1)

        agregar_columnas(posibles_variables_predictoras, columnas_importantes, cant_columnas_importantes)
        agregar_columnas(posibles_variables_predictoras, columnas_extras, cant_columnas_extras)
        return posibles_variables_predictoras

crea, entrena y luego predice un modelo con hiperparametros randomizados sobre datasets peque;os para reducir el tiempo de computo y obtener buenos hiperparametros

In [13]:
def random_validator(cantidad_modelos, posibles_cantidad_de_primera_capa, posibles_funcion_activacion, posibles_optimizador, 
    posibles_learning_rate, posibles_metricas, obtener_posibles_variables_predictoras):
    
    resultados = []
    for i in range(cantidad_modelos):

        variables_predictoras = obtener_posibles_variables_predictoras()
        cantidad_de_primera_capa = random.choice(posibles_cantidad_de_primera_capa)
        funcion_activacion = random.choice(posibles_funcion_activacion)
        optimizador = random.choice(posibles_optimizador)
        learning_rate = random.choice(posibles_learning_rate)
        metrica = random.choice(posibles_metricas)
        epoch = random.randrange(100, 300, 10)
        batch_size = random.randrange(10, 1000, 1)
        df_reducido = viviendas_train[:3000]#random.randrange(3000, 5000, 1)
        df_train, df_test = train_test_split(df_reducido.loc[:],test_size=0.2)

        print(variables_predictoras)

        print('\n','\n')
        print(i , '\n',
        variables_predictoras , '\n',
        cantidad_de_primera_capa ,'\n',
        funcion_activacion ,'\n',
        optimizador ,'\n',
        learning_rate ,'\n',
        metrica ,'\n',
        epoch ,'\n',
        batch_size ,'\n',)

        modelo = ModeloRedes(df_train, df_test, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size)
        resultados.append([modelo, modelo.testear_modelo()])
        print(modelo.testear_modelo())
    return resultados

In [14]:
#Hace 50 modelos con hiperparametros random random
def func():
    resultados = random_validator(50, posibles_cantidad_de_primera_capa, posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas,obtener_posibles_variables_predictoras)

    return resultados

In [15]:
#dado un conjunto de modelos se queda con los modelos cuyo rmse sea menor a 130000
def obtener_mejores(resultados):
    mejores = []
    for i in resultados:
        if (i[1] < 130000):
            mejores.append(i)

    return mejores

Entrenamos 150 modelos

In [16]:
mejores = []

In [17]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['longitud', 'property_surface_covered', 'barrio_Villa del Parque']

 

0 
 ['longitud', 'property_surface_covered', 'barrio_Villa del Parque'] 
 4 
 softmax 
 <function funcion_optimizadoraRMSPROP at 0x00000173954BD0D0> 
 0.01 
 mae 
 180 
 687 

19/19 [==============================] - 0s 594us/step
349125.1792993948
['longitud', 'property_surface_total', 'latitud', 'barrio_Parque Patricios', 'barrio_Constitución', 'barrio_Palermo', 'barrio_Floresta']

 

1 
 ['longitud', 'property_surface_total', 'latitud', 'barrio_Parque Patricios', 'barrio_Constitución', 'barrio_Palermo', 'barrio_Floresta'] 
 11 
 sigmoid 
 <function funcion_optimizadoraRMSPROP at 0x00000173954BD0D0> 
 0.01 
 mse 
 120 
 566 

19/19 [==============================] - 0s 634us/step
272211.6651718769
['longitud', 'property_surface_total', 'latitud', 'barrio_Villa Pueyrredón', 'barrio_Villa Lugano', 'barrio_Parque Patricios', 'barrio_Villa Real', 'barrio_Barracas', 'property_type_Casa', 'barrio_Coghlan', 'barrio_Las 

In [18]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['latitud', 'longitud', 'property_surface_covered', 'barrio_Constitución', 'barrio_Chacarita', 'barrio_Villa Soldati', 'barrio_Parque Patricios', 'barrio_Puerto Madero', 'barrio_Parque Centenario', 'barrio_Agronomía', 'barrio_Boca', 'barrio_Villa Ortuzar', 'barrio_Villa Devoto', 'property_bedrooms', 'barrio_Flores', 'barrio_Villa Urquiza', 'barrio_Villa Lugano', 'barrio_Recoleta', 'barrio_Villa Riachuelo', 'property_type_Departamento', 'barrio_Saavedra', 'barrio_San Cristobal', 'barrio_Monserrat', 'barrio_Villa Real', 'barrio_Abasto', 'barrio_Parque Chacabuco', 'property_type_PH', 'barrio_Caballito', 'barrio_Colegiales', 'barrio_Villa Crespo', 'barrio_Villa Luro', 'property_type_Casa', 'barrio_Belgrano', 'barrio_Barrio Norte', 'barrio_Catalinas', 'barrio_Barracas', 'barrio_Congreso', 'barrio_Villa General Mitre', 'barrio_Villa del Parque', 'barrio_Nuñez', 'barrio_San Nicolás', 'barrio_Once', 'barrio_Balvanera', 'barrio_Centro / Microcentro', 'barrio_Villa Santa Rita', 'barrio_Pompeya',

In [19]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_total', 'barrio_Versalles', 'barrio_Parque Patricios', 'barrio_Villa General Mitre', 'barrio_Constitución', 'barrio_Villa Ortuzar', 'barrio_Boca', 'barrio_Once', 'barrio_Mataderos', 'barrio_Palermo', 'barrio_Liniers', 'barrio_Villa Lugano', 'barrio_Velez Sarsfield', 'barrio_Pompeya', 'barrio_San Cristobal', 'barrio_Retiro', 'barrio_Parque Avellaneda', 'barrio_Paternal', 'barrio_Monserrat', 'property_rooms', 'barrio_Parque Centenario', 'barrio_Congreso', 'barrio_Villa Soldati', 'barrio_Boedo', 'barrio_San Nicolás', 'barrio_Barrio Norte']

 

0 
 ['property_surface_total', 'barrio_Versalles', 'barrio_Parque Patricios', 'barrio_Villa General Mitre', 'barrio_Constitución', 'barrio_Villa Ortuzar', 'barrio_Boca', 'barrio_Once', 'barrio_Mataderos', 'barrio_Palermo', 'barrio_Liniers', 'barrio_Villa Lugano', 'barrio_Velez Sarsfield', 'barrio_Pompeya', 'barrio_San Cristobal', 'barrio_Retiro', 'barrio_Parque Avellaneda', 'barrio_Paternal', 'barrio_Monserrat', 'property_rooms', 

In [20]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['latitud', 'barrio_Villa Devoto', 'barrio_Mataderos', 'barrio_Villa Santa Rita', 'property_type_PH', 'property_bedrooms', 'barrio_Parque Chas']

 

0 
 ['latitud', 'barrio_Villa Devoto', 'barrio_Mataderos', 'barrio_Villa Santa Rita', 'property_type_PH', 'property_bedrooms', 'barrio_Parque Chas'] 
 8 
 softmax 
 <function funcion_optimizadoraADADELTA at 0x00000173955075E0> 
 0.1 
 mse 
 290 
 581 

19/19 [==============================] - 0s 720us/step
217088.7584771631
['latitud', 'barrio_Villa Soldati', 'barrio_Villa Luro', 'barrio_Liniers', 'barrio_Congreso', 'barrio_San Nicolás', 'barrio_Parque Avellaneda', 'barrio_Nuñez', 'barrio_Coghlan', 'barrio_Parque Chacabuco', 'barrio_Villa Devoto', 'barrio_Mataderos', 'barrio_San Cristobal', 'barrio_Caballito', 'barrio_Recoleta', 'barrio_Balvanera', 'barrio_Villa Crespo', 'barrio_Floresta', 'barrio_Chacarita', 'barrio_Villa Lugano', 'barrio_Villa Ortuzar']

 

1 
 ['latitud', 'barrio_Villa Soldati', 'barrio_Villa Luro', 'barrio_Liniers', 'b

In [21]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_covered', 'longitud', 'latitud', 'barrio_San Telmo', 'barrio_Villa Crespo', 'barrio_Parque Chacabuco', 'barrio_Caballito', 'barrio_Liniers', 'barrio_Agronomía', 'barrio_Almagro', 'barrio_Once', 'property_type_Casa']

 

0 
 ['property_surface_covered', 'longitud', 'latitud', 'barrio_San Telmo', 'barrio_Villa Crespo', 'barrio_Parque Chacabuco', 'barrio_Caballito', 'barrio_Liniers', 'barrio_Agronomía', 'barrio_Almagro', 'barrio_Once', 'property_type_Casa'] 
 1 
 softmax 
 <function funcion_optimizadoraADADELTA at 0x00000173955075E0> 
 0.1 
 mse 
 140 
 40 

19/19 [==============================] - 0s 589us/step
208924.00897831886
['longitud', 'barrio_San Nicolás', 'barrio_Parque Chacabuco', 'barrio_Parque Chas', 'barrio_Pompeya', 'barrio_Caballito', 'barrio_Agronomía', 'barrio_Congreso', 'barrio_Liniers', 'barrio_Barracas', 'barrio_Villa Soldati', 'barrio_Flores', 'barrio_Saavedra', 'barrio_Abasto', 'barrio_Villa del Parque', 'property_type_PH', 'barrio_Paternal', 'bar

In [22]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['longitud', 'property_surface_covered', 'latitud', 'property_surface_total', 'barrio_Villa General Mitre', 'barrio_Barrio Norte', 'barrio_Catalinas', 'barrio_Villa Santa Rita', 'barrio_Belgrano', 'barrio_Retiro', 'barrio_Villa Soldati', 'barrio_Barracas', 'barrio_Las Cañitas', 'barrio_Balvanera', 'barrio_Villa Luro', 'barrio_Villa Real', 'barrio_San Cristobal', 'barrio_Parque Chacabuco']

 

0 
 ['longitud', 'property_surface_covered', 'latitud', 'property_surface_total', 'barrio_Villa General Mitre', 'barrio_Barrio Norte', 'barrio_Catalinas', 'barrio_Villa Santa Rita', 'barrio_Belgrano', 'barrio_Retiro', 'barrio_Villa Soldati', 'barrio_Barracas', 'barrio_Las Cañitas', 'barrio_Balvanera', 'barrio_Villa Luro', 'barrio_Villa Real', 'barrio_San Cristobal', 'barrio_Parque Chacabuco'] 
 13 
 tanh 
 <function funcion_optimizadoraRMSPROP at 0x00000173954BD0D0> 
 0.1 
 mae 
 200 
 587 

19/19 [==============================] - 0s 818us/step
321317.73378278106
['longitud', 'latitud', 'property

In [23]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['latitud', 'barrio_Liniers', 'barrio_Boedo', 'barrio_Villa General Mitre', 'barrio_Chacarita', 'barrio_Catalinas', 'barrio_Once', 'barrio_Centro / Microcentro', 'barrio_Paternal', 'barrio_Constitución', 'barrio_Coghlan', 'barrio_Saavedra', 'barrio_Villa Soldati', 'barrio_San Nicolás', 'barrio_Velez Sarsfield', 'property_rooms', 'property_type_PH', 'barrio_Barracas', 'barrio_San Cristobal', 'barrio_Villa Ortuzar', 'barrio_Villa Crespo', 'property_type_Casa', 'barrio_Flores', 'barrio_Villa Lugano', 'barrio_Floresta', 'property_type_Departamento', 'barrio_Pompeya', 'barrio_Parque Avellaneda', 'barrio_Parque Patricios', 'barrio_Monserrat', 'barrio_Caballito', 'barrio_Nuñez', 'barrio_Palermo', 'barrio_Agronomía', 'barrio_Puerto Madero', 'barrio_Abasto', 'barrio_Villa Pueyrredón', 'barrio_Las Cañitas', 'barrio_Congreso', 'barrio_Villa Urquiza', 'barrio_Tribunales', 'barrio_Boca', 'property_bedrooms', 'barrio_Barrio Norte', 'barrio_Villa Riachuelo', 'barrio_Villa Santa Rita', 'barrio_Retiro'

In [24]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_total', 'property_surface_covered', 'barrio_Coghlan', 'barrio_Constitución']

 

0 
 ['property_surface_total', 'property_surface_covered', 'barrio_Coghlan', 'barrio_Constitución'] 
 15 
 sigmoid 
 <function funcion_optimizadoraRMSPROP at 0x00000173954BD0D0> 
 0.01 
 mse 
 270 
 857 

19/19 [==============================] - 0s 914us/step
313517.0907399114
['longitud', 'property_surface_covered', 'latitud', 'property_surface_total', 'barrio_Abasto', 'property_type_Departamento', 'barrio_Villa del Parque', 'barrio_Monte Castro', 'barrio_Villa Real', 'barrio_Constitución', 'barrio_Villa General Mitre', 'barrio_Boedo', 'barrio_Once', 'barrio_Recoleta', 'barrio_Nuñez', 'barrio_Catalinas', 'barrio_Villa Devoto', 'barrio_Flores', 'barrio_Liniers', 'barrio_Coghlan', 'barrio_Villa Riachuelo', 'barrio_Floresta', 'barrio_Boca', 'barrio_Villa Santa Rita', 'property_type_Casa', 'barrio_Villa Luro', 'barrio_Villa Urquiza']

 

1 
 ['longitud', 'property_surface_covered', 'latitud

In [25]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_covered', 'barrio_Flores', 'barrio_Chacarita', 'barrio_Catalinas', 'barrio_Parque Chas', 'barrio_Caballito', 'barrio_Villa Pueyrredón', 'barrio_Puerto Madero', 'barrio_Tribunales', 'barrio_Villa Soldati', 'barrio_Paternal', 'barrio_Floresta', 'barrio_Once', 'barrio_Villa del Parque', 'barrio_Parque Avellaneda', 'barrio_Villa Santa Rita', 'barrio_Boedo', 'barrio_Villa Devoto', 'barrio_Nuñez', 'barrio_Parque Patricios', 'barrio_Belgrano', 'barrio_Palermo', 'barrio_Retiro', 'barrio_Liniers', 'barrio_Pompeya', 'barrio_Constitución', 'barrio_Monserrat', 'property_type_Casa', 'barrio_Boca']

 

0 
 ['property_surface_covered', 'barrio_Flores', 'barrio_Chacarita', 'barrio_Catalinas', 'barrio_Parque Chas', 'barrio_Caballito', 'barrio_Villa Pueyrredón', 'barrio_Puerto Madero', 'barrio_Tribunales', 'barrio_Villa Soldati', 'barrio_Paternal', 'barrio_Floresta', 'barrio_Once', 'barrio_Villa del Parque', 'barrio_Parque Avellaneda', 'barrio_Villa Santa Rita', 'barrio_Boedo', 'barri

In [26]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_total', 'latitud', 'property_bedrooms', 'barrio_Palermo', 'barrio_Chacarita', 'property_type_PH', 'barrio_Villa Ortuzar', 'barrio_Coghlan', 'property_type_Casa', 'barrio_Pompeya', 'barrio_Liniers', 'barrio_Parque Centenario', 'barrio_Monte Castro', 'barrio_Agronomía', 'barrio_Almagro', 'barrio_Puerto Madero', 'barrio_Barracas', 'barrio_Villa Santa Rita']

 

0 
 ['property_surface_total', 'latitud', 'property_bedrooms', 'barrio_Palermo', 'barrio_Chacarita', 'property_type_PH', 'barrio_Villa Ortuzar', 'barrio_Coghlan', 'property_type_Casa', 'barrio_Pompeya', 'barrio_Liniers', 'barrio_Parque Centenario', 'barrio_Monte Castro', 'barrio_Agronomía', 'barrio_Almagro', 'barrio_Puerto Madero', 'barrio_Barracas', 'barrio_Villa Santa Rita'] 
 3 
 tanh 
 <function funcion_optimizadoraNADAM at 0x00000173954FDDC0> 
 0.1 
 mae 
 160 
 284 

19/19 [==============================] - 0s 955us/step
285591.185591678
['longitud', 'barrio_Floresta', 'barrio_Villa Lugano', 'barrio_Coghlan

In [27]:

resultados = func()
mejores.append(obtener_mejores(resultados))

['property_surface_covered', 'latitud', 'longitud', 'barrio_Centro / Microcentro', 'barrio_Agronomía', 'barrio_Villa Crespo', 'barrio_Retiro', 'barrio_Villa Ortuzar', 'barrio_Once', 'barrio_Villa Urquiza', 'barrio_Abasto', 'barrio_Las Cañitas', 'barrio_San Nicolás', 'barrio_Liniers', 'barrio_Balvanera', 'barrio_Palermo', 'barrio_Parque Chacabuco', 'barrio_Parque Patricios', 'barrio_Boca', 'barrio_Coghlan', 'property_rooms', 'barrio_Monte Castro', 'barrio_Villa Devoto', 'barrio_Tribunales', 'barrio_Barrio Norte', 'barrio_Villa Pueyrredón', 'barrio_Monserrat', 'barrio_Constitución', 'barrio_Nuñez', 'barrio_Villa Lugano', 'barrio_San Telmo', 'barrio_Catalinas', 'barrio_Villa Santa Rita', 'barrio_Villa Real', 'barrio_Recoleta', 'barrio_Paternal', 'barrio_Mataderos', 'barrio_Barracas', 'barrio_Puerto Madero', 'barrio_Colegiales', 'barrio_Parque Centenario', 'barrio_Versalles', 'property_bedrooms', 'property_type_Casa', 'barrio_Villa Riachuelo', 'barrio_Pompeya', 'barrio_Belgrano', 'barrio_P

In [28]:
resultados = func()
mejores.append(obtener_mejores(resultados))

['latitud', 'barrio_Boedo', 'barrio_Almagro', 'barrio_Villa Luro', 'barrio_Monte Castro', 'barrio_Velez Sarsfield', 'barrio_Centro / Microcentro', 'barrio_Colegiales', 'barrio_Caballito', 'barrio_Constitución', 'barrio_Villa Real', 'barrio_Villa Soldati', 'barrio_Nuñez', 'barrio_Barrio Norte', 'barrio_Villa Pueyrredón', 'barrio_Balvanera', 'barrio_Pompeya', 'barrio_Villa Ortuzar', 'barrio_Saavedra', 'barrio_Recoleta', 'barrio_Villa del Parque', 'barrio_Villa General Mitre', 'barrio_San Telmo', 'barrio_Parque Chas', 'barrio_Villa Crespo', 'barrio_Barracas', 'barrio_Once', 'barrio_Belgrano', 'property_bedrooms', 'barrio_Congreso', 'barrio_Flores', 'barrio_Chacarita', 'barrio_Versalles', 'barrio_Boca', 'barrio_Floresta', 'barrio_Retiro', 'barrio_Villa Santa Rita', 'barrio_Parque Avellaneda', 'barrio_Palermo', 'barrio_Coghlan', 'property_type_Departamento', 'property_rooms', 'property_type_Casa', 'barrio_Agronomía', 'barrio_Tribunales', 'barrio_Monserrat']

 

0 
 ['latitud', 'barrio_Boedo

In [29]:
for i in mejores:
    for j in i:
        print(j)
        j[0].info()

[<__main__.ModeloRedes object at 0x00000174603C9160>, 106606.51404747745]

 


 ['property_surface_covered', 'longitud', 'property_surface_total', 'latitud', 'barrio_Agronomía', 'barrio_Catalinas', 'property_type_Departamento', 'barrio_Recoleta', 'barrio_Liniers', 'property_type_PH', 'barrio_Parque Chacabuco'] 
 1 
 softmax 
 <function funcion_optimizadoraADAMAX at 0x00000173941A4040> 
 0.01 
 mse 
 290 
 280 

[<__main__.ModeloRedes object at 0x0000017397475250>, 127315.56963532508]

 


 ['latitud', 'property_surface_covered', 'property_surface_total', 'property_type_Departamento', 'barrio_San Cristobal', 'barrio_Parque Patricios', 'barrio_Mataderos', 'barrio_Parque Avellaneda', 'barrio_Villa Pueyrredón', 'barrio_Parque Centenario', 'barrio_Agronomía', 'barrio_Parque Chacabuco', 'barrio_Puerto Madero', 'barrio_Palermo', 'barrio_Belgrano', 'property_type_Casa', 'barrio_Monte Castro', 'barrio_Nuñez', 'barrio_Catalinas', 'barrio_Villa Crespo', 'barrio_Almagro', 'barrio_Monserrat', 'barr

Entrenamos modelos con los hiperparametros obtenidos en los datasets mas chicos sobre todo el dataset

In [30]:
#reentrena un modelo con los mismo hiperparametros a ver si con una semilla mejor mejora la metrica
def reentrenar_tot(modelo):
    #modelo.info()
    mejor_modelo = ModeloRedes(
        viviendas_train, 
        viviendas_test, 
        modelo.variables_predictoras, 
        modelo.cantidad_primera_capa, 
        modelo.funcion_activacion, 
        modelo.optimizador, 
        modelo.learning_rate, 
        modelo.metrica, 
        modelo.epoch, 
        modelo.batch_size)
        
    mejor_modelo.testear_modelo()
    mejor_resultado = mejor_modelo.resultado 
   
    if mejor_resultado > 150000:
        return None

    for i in range(2):
        print('valor a vencer', mejor_resultado)
        modelo_aux = ModeloRedes(
            viviendas_train, 
            viviendas_test, 
            modelo.variables_predictoras, 
            modelo.cantidad_primera_capa, 
            modelo.funcion_activacion, 
            modelo.optimizador, 
            modelo.learning_rate, 
            modelo.metrica, 
            modelo.epoch, 
            modelo.batch_size)

        resultado_aux = modelo_aux.testear_modelo()
        print('reentrenando intento', i+1)
        resultado_aux = modelo_aux.testear_modelo()
        print(resultado_aux)
        mejor_modelo.info()
        
        if resultado_aux < mejor_resultado:
            mejor_resultado = resultado_aux
            mejor_modelo = modelo_aux 
    return [mejor_modelo, mejor_resultado]

In [31]:
tots = []

In [32]:
for i in mejores:
    for j in i:
        tots.append(reentrenar_tot(j[0]))

581/581 [==============================] - 0s 419us/step
valor a vencer 132663.7845502574
581/581 [==============================] - 0s 444us/step
reentrenando intento 1
581/581 [==============================] - 0s 411us/step
123863.62774056353

 


 ['property_surface_covered', 'longitud', 'property_surface_total', 'latitud', 'barrio_Agronomía', 'barrio_Catalinas', 'property_type_Departamento', 'barrio_Recoleta', 'barrio_Liniers', 'property_type_PH', 'barrio_Parque Chacabuco'] 
 1 
 softmax 
 <function funcion_optimizadoraADAMAX at 0x00000173941A4040> 
 0.01 
 mse 
 290 
 280 

valor a vencer 123863.62774056353
581/581 [==============================] - 0s 416us/step
reentrenando intento 2
581/581 [==============================] - 0s 413us/step
143008.59404716233

 


 ['property_surface_covered', 'longitud', 'property_surface_total', 'latitud', 'barrio_Agronomía', 'barrio_Catalinas', 'property_type_Departamento', 'barrio_Recoleta', 'barrio_Liniers', 'property_type_PH', 'barrio_Parq

In [33]:
print(tots)

[[<__main__.ModeloRedes object at 0x0000017458718BB0>, 123863.62774056353], [<__main__.ModeloRedes object at 0x00000173AA4B9A30>, 112868.26922674004], None, None, [<__main__.ModeloRedes object at 0x000001744F983AC0>, 115992.32327733895], None, None, [<__main__.ModeloRedes object at 0x0000017394EF3B80>, 121356.77089978676], [<__main__.ModeloRedes object at 0x0000017447CCB850>, 139222.69493160778], [<__main__.ModeloRedes object at 0x000001741C742370>, 131970.40261266343], None, None, None, [<__main__.ModeloRedes object at 0x0000017460308670>, 114197.17095412056], [<__main__.ModeloRedes object at 0x00000173A2CCEAF0>, 110623.7470332297], None, [<__main__.ModeloRedes object at 0x000001740F8934F0>, 123144.49091281014], [<__main__.ModeloRedes object at 0x00000173B30FC220>, 114450.11139009752]]


In [34]:
mejor_modelo = tots[0]
for i in tots:
    if (i != None) and (i[1] < mejor_modelo[1]):
        mejor_modelo = i



In [35]:
mejor_modelo[0].testear_modelo()

581/581 [==============================] - 1s 1ms/step


110623.7470332297

In [36]:
joblib.dump(mejor_modelo[0], './Models/mejor_modelo_ultima_tanda') 

INFO:tensorflow:Assets written to: ram://4983970d-4f26-43db-9e2e-d4e049c6daef/assets


['./Models/mejor_modelo_ultima_tanda']

In [37]:
#mejor_modelo = joblib.load('./Models/mejor_modelo_ultima_tanda')

El mejor modelo obtenido hasta ahora con estas tecnicas es:

In [38]:
mejor_modelo = joblib.load('./Models/modelo_regresion_103k')

ReadError: file could not be opened successfully

In [ ]:
#ModeloRedes(df_train, df_test, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size)

no_te_queremos_perder_hermoso = ModeloRedes(
    viviendas_train, 
    viviendas_test, 
    mejor_modelo.variables_predictoras, 
    mejor_modelo.cantidad_primera_capa, 
    mejor_modelo.funcion_activacion, 
    mejor_modelo.optimizador, 
    mejor_modelo.learning_rate, 
    mejor_modelo.metrica, 
    mejor_modelo.epoch, 
    mejor_modelo.batch_size)

AttributeError: 'list' object has no attribute 'testear_modelo'

In [ ]:
no_te_queremos_perder_hermoso.testear_modelo()

hasat aca corre

In [ ]:
mejor_modelo.plotear()

In [ ]:
# Performance
performance_test = pd.DataFrame({'Valor Real': y_test_knn,
                            'Prediccion': y_pred_test_knn.round(),
                            'Error': y_test_knn - y_pred_test_knn.round()})
# View

performance_test.head()

In [ ]:
reentrenado = ModeloRedes(viviendas, 
    mejor_modelo.variables_predictoras, 
    mejor_modelo.cantidad_primera_capa, 
    mejor_modelo.funcion_activacion, 
    mejor_modelo.optimizador, 
    mejor_modelo.learning_rate, 
    mejor_modelo.metrica, 
    mejor_modelo.epoch, 
    mejor_modelo.batch_size, 
    viviendas.size)

In [ ]:
mejor_modelo.x_train.shape

In [ ]:
mejor_modelo.y_test

In [ ]:
datos_test = pd.DataFrame(mejor_modelo.x_test)
b = pd.DataFrame(mejor_modelo.y_test)
datos_test.join(b)

In [ ]:
mejor_modelo.predecir(datos_test)

In [ ]:
test = pd.concat(mejor_modelo.x_test, mejor_modelo.x_train)
mejor_modelo.predecir(mejor_modelo.x_test)

In [ ]:
mejor_modelo = joblib.load('./Models/modelo_regresion_103k')
        
y_pred=mejor_modelo.modelo.predict(mejor_modelo.x_train)

aux = []
for i in range(len(y_pred)):
     aux.append(y_pred[i][0])  
"""
y_pred = pd.Series(aux)"""
y_pred = aux

performance_train = pd.DataFrame({'Valor Real': mejor_modelo.y_train,
                            'Prediccion': y_pred,
                            'Error': mejor_modelo.y_train - y_pred
                             })

performance_train.head()
performance_train.plot.scatter(x="Valor Real", y="Prediccion", s=10, c='tab:blue').set(title="Valor Real vs Predicción",xlabel='Valor Real',ylabel='Prediccion')

In [ ]:
performance_train.head()

In [ ]:
print('Metrica rmse', mejor_modelo.testear_modelo())

mejor_modelo.info()

In [ ]:
#joblib.dump(model_tot, './Models/red_regresion_108k_tot') 